<a href="https://colab.research.google.com/github/Tsirwed/Tsirwed/blob/main/sacco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  SACCO Loan Eligibility Evaluation

This notebook models a SACCO (Savings and Credit Cooperative) that evaluates members for loan approval.
It demonstrates object-oriented programming, exception handling, and currency conversion from UGX to USD.


In [ ]:
from abc import ABC, abstractmethod
import requests
from requests.exceptions import RequestException


In [ ]:
class Member(ABC):
    """
    Abstract Base Class representing a SACCO member.
    Attributes:
        name (str): Member's name
        id_no (str): Member's ID number
        savings_balance (float): Member's savings balance
    """

    def __init__(self, name, id_no, savings_balance):
        self.name = name
        self.id_no = id_no
        self.savings_balance = savings_balance

    @abstractmethod
    def loan_eligibility(self):
        """Calculate loan eligibility based on savings balance."""
        pass

    @staticmethod
    def get_exchange_rate():
        """
        Fetch USD -> UGX exchange rate using API.
        Returns fallback rate if network or JSON errors occur.
        """
        url = "https://api.exchangerate.host/latest?base=USD&symbols=UGX"
        try:
            response = requests.get(url, timeout=5)
            response.raise_for_status()
            data = response.json()
            rate = data["rates"]["UGX"]
            return float(rate)
        except (RequestException, KeyError, ValueError) as e:
            print(f"[Error fetching exchange rate]: {e}")
            fallback_rate = 3700.0
            print(f"Using fallback rate: 1 USD = {fallback_rate} UGX")
            return fallback_rate

    def display_loan(self):
        """
        Display loan eligibility in UGX and USD.
        Handles exceptions during calculation or display.
        """
        try:
            ugx_eligibility = self.loan_eligibility()
            rate = Member.get_exchange_rate()
            usd_eligibility = ugx_eligibility / rate
            print(f"Member: {self.name} | ID: {self.id_no}")
            print(f"Loan Eligibility: UGX {ugx_eligibility:,.2f} | USD {usd_eligibility:,.2f}\n")
        except Exception as e:
            print(f"[Error calculating/displaying loan]: {e}")


In [ ]:
class TraderMember(Member):
    """Trader member: loan eligibility = 4 × savings balance"""
    def loan_eligibility(self):
        return self.savings_balance * 4

class FarmerMember(Member):
    """Farmer member: loan eligibility = 6 × savings balance"""
    def loan_eligibility(self):
        return self.savings_balance * 6


In [ ]:

trader = TraderMember("Alice", "T123", 500_000)
farmer = FarmerMember("Bob", "F456", 300_000)

trader.display_loan()
farmer.display_loan()


[Error fetching exchange rate]: 'rates'
Using fallback rate: 1 USD = 3700.0 UGX
Member: Alice | ID: T123
Loan Eligibility: UGX 2,000,000.00 | USD 540.54

[Error fetching exchange rate]: 'rates'
Using fallback rate: 1 USD = 3700.0 UGX
Member: Bob | ID: F456
Loan Eligibility: UGX 1,800,000.00 | USD 486.49



In [ ]:
import json

def save_loans_to_json(members, filename="loan_eligibility.json"):
    """
    Save loan eligibility data for members to a JSON file.
    """
    data_to_save = []
    rate = Member.get_exchange_rate()  # get exchange rate once

    for member in members:
        try:
            ugx_eligibility = member.loan_eligibility()
            usd_eligibility = ugx_eligibility / rate
            data_to_save.append({
                "name": member.name,
                "id_no": member.id_no,
                "savings_balance": member.savings_balance,
                "loan_eligibility_UGX": ugx_eligibility,
                "loan_eligibility_USD": round(usd_eligibility, 2)
            })
        except Exception as e:
            print(f"[Error processing member {member.name}]: {e}")

    try:
        with open(filename, "w") as f:
            json.dump(data_to_save, f, indent=4)
        print(f"Loan eligibility data saved to '{filename}'")
    except Exception as e:
        print(f"[Error saving JSON]: {e}")


In [ ]:
from IPython.display import FileLink

members_list = [trader, farmer]
save_loans_to_json(members_list, "loan_eligibility.json")

FileLink("loan_eligibility.json")


[Error fetching exchange rate]: 'rates'
Using fallback rate: 1 USD = 3700.0 UGX
Loan eligibility data saved to 'loan_eligibility.json'


/content/loan_eligibility.json